# ДЗ №2, Волжина Елена
[Выравнивание последовательностей](https://compscicenter.ru/learning/assignments/24199/)


Напишите программу, получающую на вход две последовательности, и выдающую их оптимальное выравнивание (имеющее максимальный вес).
* Вес совпадения = 1.
* Вес несовпадения = -m (задается вами).
* Вес гэпа = -d (задается вами).

In [1]:
from collections import defaultdict

M = 1    # mismatch
D = 1    # gap
string1, string2 = "GATTACA", "GCATGCU"

In [2]:
def align(s1, s2):
    assert s1 and s2, "one of strings is empty!"
    
    n, m = len(s1), len(s2)
    sources = defaultdict(lambda: defaultdict(list))
    weights = [[None for _ in range(m + 1)] for _ in range(n + 1)]
    
    # first row
    for n_col in range(m + 1):
        weights[0][n_col] = -n_col * D
        sources[0][n_col] = [(0, n_col - 1)]
        
    # first column
    for n_row in range(n + 1):
        weights[n_row][0] = -n_row * D
        sources[n_row][0] = [(n_row - 1, 0)]
    
    # dynamically fill the rest
    def process(n_row, n_col):
        matched = s1[n_row - 1] == s2[n_col - 1]
        ways = [((n_row - 1, n_col - 1), 1 if matched else -M),  # (mis)match
                ((n_row - 1, n_col), -D),   # move down
                ((n_row, n_col - 1), -D)]   # move right
        for (old_n_row, old_n_col), diff in ways:
            prev_value = weights[old_n_row][old_n_col]
            new_value = prev_value + diff
            
            if weights[n_row][n_col] is None or weights[n_row][n_col] < new_value:
                weights[n_row][n_col] = new_value
                sources[n_row][n_col] = [(old_n_row, old_n_col)]
            elif weights[n_row][n_col] == new_value:
                sources[n_row][n_col].append((old_n_row, old_n_col))
    
    for k in range(1, min(n, m) + 1):
        process(k, k)
        for n_row in range(k + 1, n + 1):
            process(n_row, k)
        for n_col in range(k + 1, m + 1):
            process(k, n_col)
        
    return weights, sources

In [3]:
alignment_weights, alignment_sources = align(string1, string2)

In [4]:
print("WEIGHTS MATRIX")
print("\n".join(" ".join("{: >3}".format(x) for x in row) 
                for row in alignment_weights))

WEIGHTS MATRIX
  0  -1  -2  -3  -4  -5  -6  -7
 -1   1   0  -1  -2  -3  -4  -5
 -2   0   0   1   0  -1  -2  -3
 -3  -1  -1   0   2   1   0  -1
 -4  -2  -2  -1   1   1   0  -1
 -5  -3  -3  -1   0   0   0  -1
 -6  -4  -2  -2  -1  -1   1   0
 -7  -5  -3  -1  -2  -2   0   0


In [5]:
def restore_paths(sources, n_row, n_col):
    if sources[n_row][n_col]:
        result = []
        for source in sources[n_row][n_col]:
            paths = restore_paths(sources, *source)
            result.extend([p + [(n_row, n_col)] for p in paths])
        return result
    else:
        return [[]]


alignment_paths = restore_paths(alignment_sources, len(string1), len(string2))
alignment_paths

[[(0, 0), (1, 1), (1, 2), (2, 3), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7)],
 [(0, 0), (1, 1), (1, 2), (2, 3), (3, 4), (4, 4), (5, 5), (6, 6), (7, 7)],
 [(0, 0), (1, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 5), (6, 6), (7, 7)]]

In [6]:
for path in alignment_paths:
    res1, res2 = [], []
    for (r1, c1), (r2, c2) in zip(path, path[1:]):
        if r1 + 1 == r2 and c1 + 1 == c2:
            # mis(match)
            res1.append(string1[r2 - 1])
            res2.append(string2[c2 - 1])
        elif r1 + 1 == r2 and c1 == c2:
            # down
            res1.append(string1[r2 - 1])
            res2.append("-")
        else:
            # right
            res1.append("-")
            res2.append(string2[c2 - 1])
    print(res1, "\n", res2, "\n\n", sep="")

['G', '-', 'A', 'T', 'T', 'A', 'C', 'A']
['G', 'C', 'A', '-', 'T', 'G', 'C', 'U']


['G', '-', 'A', 'T', 'T', 'A', 'C', 'A']
['G', 'C', 'A', 'T', '-', 'G', 'C', 'U']


['G', '-', 'A', 'T', 'T', 'A', 'C', 'A']
['G', 'C', 'A', 'T', 'G', '-', 'C', 'U']


